# Notebook Preamble

## IPython Magic

In [ ]:
%load_ext autoreload
%autoreload 3

## Notebook Imports

In [ ]:
# 3rd Party Imports:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import matplotx
import numpy as np
import duckdb
import sqlalchemy as sa
from dagster import AssetKey
from pathlib import Path
import os

# Local Imports
import pudl
from pudl.etl import defs
from pudl.output.pudltabl import PudlTabl
from pudl.workspace.setup import PudlPaths

logger = pudl.logging_helpers.get_logger("pudl")

## Visualization Settings

In [ ]:
%matplotlib inline

In [ ]:
matplotlib.rcParams["figure.figsize"] = (10, 6)
matplotlib.rcParams["figure.dpi"] = 150
matplotlib.style.use(matplotx.styles.onedark)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_colwidth", 1000)

## Grab the Data

In [ ]:
def get_table(table: str):
    return defs.load_asset_value(AssetKey(table))

def get_parquet(table: str) -> pd.DataFrame:
    return pd.read_parquet(Path(os.environ["PUDL_OUTPUT"]) / f"parquet/{table}.parquet")

## Splitting Company Info

### Company data
- 4 of the 5 tables that can be pulled out of this data seem to pertain to companies -- static or slowly changing dimensions.
- `company_data` and `filing_values` have a 1:1 mapping and can be merged based on their index without losing any information.
- There are 386 instances in which `business_address` does not have an index value that corresponds to company + filing data.
- There are 22612 instances in which `mail_address` does not have an index value that corresponds to company + filing data.
- All orphaned addresses have different values for `filer_count` and `company_information_block_count`
- All `company_data` and `filing_values` records have matching `filer_count` and `company_information_block_count` values.
- We can build a nice 400K row table with modest numbers of nulls by left-merging the address tables onto the (company data + filing values) table 
- This means about 23K addresses are not included in the final table. However, it isn't clear what company or filing those addresses ought to pertain to.
- The only clear identifier(s) for the orphaned addresses are the CIK in the filename, but that will generally pertain to the main filer, which may have a different address.
- A quick visual inspection suggests that many of the orphaned addresses are additional instances or forms of addresses already included in the table.
- What is an appropriate natural primary key for this table?
- The `report_date` field is not independent -- it could be derived from the date associated with the filing's filename / file ID.
- The `filer_count` and `company_information_block_count` columns are always equal after this merge, so only one integer column can be kept.
- The filename + one of the counter columns would result in a unique PK. What would that mean conceptually? Master filing + Nth company filing identified in that master filing?
- There's a directly reported CIK in the company data, but also a "master filer" CIK that can be extracted from the filename.
- Having both the master filer and the company CIKs in the same table might be useful, and could indicate whether the company is a subsidiary.
- The filename is mostly for provenance -- where did this data come from.
- If the reported data fields (address, TIN, name, fiscal year, etc.) are truly identical, then we don't lose any information about the companies by deduplicating them.

### Filing data
- There's also filing-level data that's associatd with the company data.
- On cursory inspection, it seems like a lot of the duplicate values that show up when trying to create a natural primary key if we concatenate the filing data with the company data are actually due to the filing data.
- If we separate the filing data out we may get a granular filing history that's separate from the less variable company data?
- The same SEC filing number can show up many times because amendments and the filings they amend share the same filing number.

### Company naming history
- There's a kind of "sub-table" that's associated with the company data, which describes the history of the company names.
- The whole history seems to be reported in each filing, so there's lots of duplication.
- Alone, this table only has the "former" names and the dates of changes away from those former names.
- Being able to merge in the report date, filename, current company name and CIK from the company table will make this table more legible / useful.
- With all of that information there will be a ton of duplication, but also probably lots of useful ways to deduplicate it in a subsequent step.
- Maybe this is a raw or intermediate table that gets handed off for subsequent processing.


## To Do List

- Integrate company info data cleaning / standardization into `core_sec10k__quarterly_company_information` asset.
- Update company info resource definition to reflect current columns / PK
- Break out an intermediate `_core_sec10k__changelog_company_name` table that does just the data cleaning.
- Move asset definitions from `pudl_models` module into `transform/sec10k.py` and `output/sec10k.py` modules.
- Update the `out_sec10k__quarterly_company_information` and `out_sec10k__changelog_company_name` assets & resource definitions.
- Add a `source_url` field that links directly to the SEC files into the output tables.
- Switch back to using `pudl_io_manager` for the company information tables.
- Update and expand SEC 10-K table descriptions.
- Pull `report_date` associated with each `filename_sec10k` into the output tables.
- Update the master filings table to be consistent with the company info changes

## Questions for Zach about header extraction

- WTF is the relationship between `filer_count` and `company_information_block_count`
- Why is it the case that only **most** records can be concatenated based on filename, filer count, report date, and company info block?

In [ ]:
company_info = get_table("core_sec10k__quarterly_company_information")

#file_sec10k = get_table("core_sec10k__quarterly_filings")
#own_sec10k = get_table("core_sec10k__quarterly_exhibit_21_company_ownership")
#out_sec10k = get_table("out_sec10k__parents_and_subsidiaries")
#name_change_sec10k = get_table("core_sec10k__changelog_company_name")

In [ ]:
company_info.info()

In [ ]:
# TIN regex pattern
tin_pattern = r"\b\d{2}-\d{7}\b"
assert company_info["taxpayer_id_irs"].str.match(tin_pattern).all()

# ZIP Code regex pattern
zip5_pattern = r"\d{5}"
assert company_info["business_zip_code"].str.match(zip5_pattern).all()
assert company_info["mail_zip_code"].str.match(zip5_pattern).all()

zip4_pattern = r"\d{4}"
assert company_info["business_zip_code_4"].str.match(zip4_pattern).all()
assert company_info["mail_zip_code_4"].str.match(zip4_pattern).all()

valid_sec10k = [
    "10-k",
    "10-k/a",
    "10-k405",
    "10-k405/a",
    "10-kt",
    "10-kt/a",
    "10-ksb",
    "10-ksb/a",
]
# All the records with an pd.NA values for `sec10k_type` seem to be failing
# But I thought NA values where fine and not in conflict with enum constraints?
# assert company_info["sec10k_type"].isin(valid_sec10k).all()
company_info.loc[company_info["sec10k_type"].isna()]

valid_sec_act = ["1934 act"]

company_info.loc[~company_info["sec_act"].isin(valid_sec_act)]

In [ ]:
assert False

## SEC 10-K Ownership Information

* We know what file each row came from, but in many cases there are multiple companies referenced in individual 10-K files. How do we decide which one of those companies should be considered the parent? I.e. who owns the `fraction_owned`?
* There seems to be a CIK embedded in the filename. Is that important for linking this data back to the company info table? I guess we can also go indirectly to CIK via the filename listed here and in the filings table.

In [ ]:
own_sec10k = get_parquet("core_sec10k__quarterly_exhibit_21_company_ownership")
display(own_sec10k.info())
own_sec10k.sample(10)

## SEC 10-K Parent / Subsidiary Table

* This is a denormalized association table that relates parents to subsidiaries.
* It should probably correspond to a very skinny normalized association table that describes just the associations using IDs, without any of the data columns, but which can be used as the backbone for merging in data from one or more other tables that contain the specific data about the parent and subsidiary companies.
* How many different kinds of companies are there, and how do they relate to each other? Can/Should the parent and subsidiary companies be described using the same data structure?
* Every column in this table should be named such that it is clear whether the attribute is associated with the parent or the subsidiary company, since there are two sets of basically similar information in the table side-by-side, e.g. with a `parent_` and `subsidiary_` prefix.
* Based on the other tables it seems like there might be as many as 3 different kinds of companies we're dealing with, or about which we might have different kinds and amounts of information.
  * The (master filer?) company listed in the `core_sec10k__quarterly_filings` table.
  * The individual companies listed in un-normalized sub-tables of `core_sec10k__company_information`.
  * The subsidiary companies in `core_sec10k_quarterly_exhibit_21_company_ownership` about which may only have a name, or at best a name, location, and fraction of that company that's owned by some parent.
* Maybe the master filers / individual companies can be described using the same structure because that information is ultimately available for all SEC 10-K filers?
* Any company that we're pulling from the filing or company info tables will have a CIK associated with it so we can probably use that to look up (time varying) information about the company somehow.
* The raw company info that we're pulling Exhibit 21 initially won't be guaranteed to have anything but a name (and a filename / report date), but then we use that to do some entity matching, and create an association table between the Exhibit 21 companies and other SEC 10-K filers.
* There should be an Exhibit 21 company table that has its own primary key that's hopefully eventually associated with one of the real SEC 10-K filer records.
* Because ownership can change over time, with subsidiaries being sold off to other companies, it seems like we have to maintain a sense of time variability in these associations. Otherwise the data won't be useful for answering questions like "In 2020Q2 when this dodgy transaction took place between companies A and B, where they owned by the same parent company?"
* What does it mean if in a given year a subsidiary shows up in Exhibit 21 for a parent company, and in the subsequent year it does **not** show up in Exhibit 21 for that same parent company? It seems like that should imply that it's no longer owned by the parent. Is there a reason why we wouldn't be able to say that?
* Once the entity matching has been done within the SEC 10-K data, connecting the sparse information we have about many subsidiary companies back to fuller information about the SEC 10-K filers, then we have a table of company information we can try to match against the EIA utility data.

### Company Tables

* SEC 10-K filers (lots of columns, PK is probably CIK?)
* Exhibit 21 Subsidiaries (very few columns, PK is probably messy/name-based, or an auto-incrementing pseudo-key)
* EIA Utilities (lots of columns from PUDL)
* Normalized (skinny) association table that links SEC 10-K filers and Exhibit 21 Subsidiaries, based on record linkage.
* Denormalized table that merges in all relevant company-level attributes from both SEC 10-K filers and Exhibit 21 Subsidiaries tables
* Many companies will show up only as one or the other, some as both, so this denormalized table would have a lot of null fields.
* Right now it seems like `company_id_sec10k` is the column that represents the primary key of this merged company table, but it's a mix of CIKs and concatenated values from the Exhibit 21 table which is a messy mix that can lead to confusion (we were bitten by a similar pattern in the EIA utility and/or BA IDs in the FERC-714 data).
* This denormalized SEC 10-K table seems like the thing that would have been used in record linkage against the EIA utilities, since it has the most SEC based company information? In which case there would also potentially be a `utility_id_eia` column for use connecting the SEC 10-K companies to the EIA utilities.

### SEC 10-K Filer (entity) Table

* `central_index_key`
* Are there any other truly permanent company fields linked to the CIK?

### SEC 10-K Filer (time-varying) company info table

* `central_index_key` (PK)
* `report_date` (PK)
* Any other time-varying, per-company fields that fall out of the normalization of `core_sec10k__quarterly_company_information`

### Exhibit 21 Company (entity) Table

Notionally this would contain any permanent attributes of the companies observed as subsidiaries in Exhibit 21. Since that table is almost unstructured, this one is probably vestigal and doesn't need to exist.

* `company_id_ex21`
* ???

### Exhibit 21 (time-varying) Company Table

All the unique **company** information that's been extracted from Exhibit 21 attachements. Note that this doesn't include the ownership fraction, which ends up in a table that is about the data that's unique to the parent-subsidiary relationship.

* `company_id_ex21` (PK, auto-incrementing integer pseudo-key?)
* `report_date` (data)
* `parent_central_index_key` (data, extracted from `filename_sec10k`?)
* `subsidiary_company_name` (data)
* `subsidiary_company_location` (data)
* `filename_sec10k` (data)

### SEC 10-K Company (Filer and/or Subsidiary) Association Table

Links companies that have been observed as both SEC 10-K filers and Exhibit 21 subsidiaries, based on statstical record linkage.  There's no explicit time variability in this table, but the fact that some companies only show up in some years means there's an implicity time dependence, which will show up in other tables with `report_date` columns.

* `company_id_sec10k` (PK)
* `company_id_ex21` (PK)
* `central_index_key` (PK)

### Parent-subsidiary association table

A normalized table that contains just the information that is unique to the association between a parent and subsidiary company pair. This can be used to merge together other pieces of information about the parent and/or subsidiary to create useful denormalized output table. Each row says "On this date, this parent owned this fraction of this subsidiary."

* `report_date` (PK)
* `parent_company_id_sec10k` (PK)
* `subsidiary_company_id_sec10k` (PK)
* `fraction_owned` (data)

### SEC 10-K Company Output Table

A wide, human-readable denormalized table that merges together all of the relevant information about a company that we have extracted from the SEC 10-K filings, whether that information comes from the core SEC 10-K filings, Exhibit 21 attachments, or EIA Utilities data we have in PUDL.

* `company_id_sec10k` (PK)
* `report_date` (PK)
* `central_index_key` (data)
* `company_id_ex21` (data)
* `utility_id_eia` (data)
* Other data columns to be selected.